In [ ]:
!pip install -U -q PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_list = drive.ListFile({'q': "'1DIF8C7O9SUS3puW3NHXymKa2Nz3GG8L7' in parents and trashed=false"}).GetList()
for file1 in file_list:
    if file1['title'] == 'raw_tweet.zip':
        preprocessorFile = drive.CreateFile({'id': file1['id']})
        preprocessorFile.GetContentFile('raw_tweet.zip')
        print('title: %s, id: %s is downloaded' % (file1['title'], file1['id']))
        break


In [ ]:
!unzip raw_tweet.zip

In [ ]:
!rm raw_tweet.zip

In [ ]:
import os
import json
import tensorflow_hub as hub
import pandas as pd
from tqdm import tqdm
import numpy as np
import random

In [ ]:
if not os.path.exists('preprocessed/GMRE'):
    os.mkdir('preprocessed/GMRE')

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
empty_embedding = embed(['']).numpy()[0]

In [ ]:
for fname in ['train_timestamp.csv', 'test_timestamp.csv', 'valid_timestamp.csv']:
    timestamps_df = pd.read_csv(fname)
    tweets_text = []
    print(f'{fname} stage 1 in process', flush=True)
    stage_max_tweet = 0
    limit_max_tweet = 10
    for i, allow_date in enumerate(tqdm(timestamps_df['Date'].values)):
        tweets_text.append([])
        for j, stock_name in enumerate(sorted(os.listdir('preprocessed'))):
            if allow_date in sorted(os.listdir(f'preprocessed/{stock_name}')):
                with open(f'preprocessed/{stock_name}/{allow_date}') as f:
                    lines = f.readlines()
                    if len(lines) > limit_max_tweet:
                        lines = random.sample(lines,limit_max_tweet)
                    tweets = [' '.join(json.loads(tweet).get('text')) for tweet in lines]
                embeddings = embed(tweets)
                tweets_text[-1].append(embeddings.numpy())
                if len(lines) < limit_max_tweet:
                    need = limit_max_tweet - len(lines)
                    tweets_text[i][j] = np.vstack([tweets_text[i][j],np.array([empty_embedding]*need)])
            else:
                tweets_text[-1].append(np.array([empty_embedding]*limit_max_tweet))
    tweets_text = np.array([np.array(i) for i in tweets_text])
    print(tweets_text.shape)

    root = 'tweet_output'
    if not os.path.exists(root):
        os.mkdir(root)
    filename = os.path.join(root, fname.split('_')[0] + '_tweet.npy')
    with open(filename, 'wb') as fn:
        np.save(fn, tweets_text)
        print('File saved')

In [ ]:
!ls -l tweet_output/

In [ ]:
stock_key_seq = np.array(sorted(os.listdir('preprocessed')))
with open('tweet_output/stock_key_seq.npy', 'wb') as fn:
    np.save(fn, stock_key_seq)
    print('File saved')

In [ ]:
!zip -r tweet_output.zip tweet_output